In [1]:
import pandas as pd

In [6]:
camas = pd.read_csv( './data/hospitales_bogota.csv', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': int, 'Camas habilitadas': int, 'Camas disponibles': int } )

In [7]:
camas.dtypes

IPS                        object
Servicio                   object
Grupo                      object
Camas Ocupadas              int64
Camas habilitadas           int64
Camas disponibles           int64
Porcentaje de Ocupación    object
Fecha                      object
dtype: object

In [8]:
camas

,IPS,Servicio,Grupo,Camas Ocupadas,Camas habilitadas,Camas disponibles,Porcentaje de Ocupación,Fecha
0,Públicas,Hospitalización General,Neonatal,98,138,40,71%,15/05/2020
1,Públicas,Hospitalización General,Pediátrico,113,400,287,28%,15/05/2020
2,Públicas,Hospitalización General,Adulto,703,1188,483,59%,15/05/2020
3,Públicas,Unidad de Cuidado Intermedio,Neonatal,98,126,28,78%,15/05/2020
4,Públicas,Unidad de Cuidado Intermedio,Pediátrico,6,14,8,43%,15/05/2020
5,Públicas,Unidad de Cuidado Intermedio,Adulto,49,72,23,68%,15/05/2020
6,Públicas,Unidad de cuidado intensivo,Neonatal,58,68,10,85%,15/05/2020
7,Públicas,Unidad de cuidado intensivo,Pediátrico,13,29,16,45%,15/05/2020
8,Públicas,Unidad de cuidado intensivo,Adulto,77,137,60,56%,15/05/2020
9,Privadas,Hospitalización General,Neonatal,102,149,47,68%,15/05/2020


In [16]:
camas.loc[ camas[ 'Servicio' ] != 'Unidad de cuidado intensivo' ].groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).sum()

Camas Ocupadas       5674
Camas habilitadas    9103
Camas disponibles    3450
dtype: int64

In [15]:
camas.loc[ camas[ 'Servicio' ] == 'Unidad de cuidado intensivo' ].groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } )

,Camas Ocupadas,Camas habilitadas,Camas disponibles
Servicio,,,
Unidad de cuidado intensivo,763,1406,648
